# Prediction of Volatiltiy with Deep Learning and GARCH model

This notebook aims to predict market volatility using machine learning integrated with an econometrics model.

Data from ```'HKEX', 'NYSE', 'NASDAQ', 'AMEX'``` during ```1998-01-01 to 2021-09-07``` was employed.

## Volatiltiy

The volatility $\sigma$, of a stock is a measure of our uncertainty about the returns provided by the stock. It can be defined as the standard deviation of the return provided by the stock. Volatility can be measured in several ways:
1. Historical volatility is calculated from historical data of a stock price.
2. Implied volatility is calculated from prices observed in the market.

# Setup

## Import Libraries

In [12]:
import os
import sys
import mysql.connector

import pandas as pd
import numpy as np
import math
import timeit
import warnings

# Environment variables
from dotenv import load_dotenv
load_dotenv("mysql.env")

# Visualization + diagnositic
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox, het_arch
from scipy.stats import probplot, shapiro

warnings.filterwarnings('ignore')

import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs

# Model
from arch import arch_model
from arch.__future__ import reindexing


# Performance
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

print('Machine: {} {}\n'.format(os.uname().sysname,os.uname().machine))
print(sys.version)

Machine: Darwin x86_64

3.8.12 | packaged by conda-forge | (default, Sep 16 2021, 01:59:00) 
[Clang 11.1.0 ]


# List of Stocks and ETFs
Provided by Thomas Choi.

In [90]:
stock_list = pd.read_csv("stocks_and_etfs/stock_list.csv")
etf_list = pd.read_csv("stocks_and_etfs/etf_list.csv")

In [141]:
stock_symbol = stock_list.iloc[8,0]
stock_symbol

'DKNG'

## MySQL connection
Choosing one stock from SQL query to reduce query time.

In [ ]:
HOST=os.environ.get("HOST")
PORT=os.environ.get("PORT")
USER=os.environ.get("USER")
PASSWORD=os.environ.get("PASSWORD")

try: 
    conn = mysql.connector.connect(
        host=HOST,
        port=PORT,
        user=USER,
        password=PASSWORD,
        database="GlobalMarketData"
    )
    query = f"SELECT Date, Close, Open, High, Low, Volume from histdailyprice3 WHERE Symbol='{stock_symbol}';"
    histdailyprice3 = pd.read_sql(query, conn)
    conn.close()
except Exception as e:
    conn.close()
    print(str(e))

# Load Data

In [ ]:
df = histdailyprice3.copy()
df.set_index("Date", drop=True, inplace=True)
df.head()

## Closing Price

In [ ]:
df['Close'].plot(title=f'Closing Price from 1998-01-01 to 2021-09-07')
plt.show()

## Calculate Daily Returns
The formula for return is the following, where $u$ denotes return and $S$ denotes the price of an underlying asset:
$$ u_n = \frac{S_n - S_{n-1}}{S_{n-1}} $$

In [ ]:
df['pct_change'] = 100 * df['Close'].pct_change()

In [ ]:
threshold = 2

df.dropna(inplace=True)
fig = plt.figure()
fig.set_figwidth(12)
plt.plot(abs(df['pct_change']), label = 'Percent change')
plt.axhline(y=threshold, color='g', linestyle='-')
plt.title('Percent change in Closing price')
plt.show()

## Calculate daily, monthly, and annual volatitily

In [ ]:
daily_volatility = df['pct_change'].std()
print('Daily volatility: ', '{:.2f}%'.format(daily_volatility))

monthly_volatility = math.sqrt(21) * daily_volatility
print ('Monthly volatility: ', '{:.2f}%'.format(monthly_volatility))

annual_volatility = math.sqrt(252) * daily_volatility
print ('Annual volatility: ', '{:.2f}%'.format(annual_volatility ))

## ACF and PACF

In [ ]:
# create acf plot
plot_acf(df["pct_change"], lags=10)
plt.show()

In [ ]:
# create pacf plot
plot_pacf(df["pct_change"], lags=10, method='ywm')
plt.show()

## Ljung-Box Test

Ljung-Box is a test for autocorrelation that we can use in tandem with our ACF and PACF plots. The Ljung-Box test takes our data, optionally either lag values to test, or the largest lag value to consider, and whether to compute the Box-Pierce statistic. Ljung-Box and Box-Pierce are two similar test statisitcs, $Q$ , that are compared against a chi-squared distribution to determine if the series is white noise. We might use the Ljung-Box test on the residuals of our model to look for autocorrelation, ideally our residuals would be white noise.

- $H_o$ : The data are independently distributed, no autocorrelation.
- $H_a$ : The data are not independently distributed; they exhibit serial correlation.
The Ljung-Box with the Box-Pierce option will return, for each lag, the Ljung-Box test statistic, Ljung-Box p-values, Box-Pierce test statistic, and Box-Pierce p-values.

If  p<α  (0.05) we reject the null hypothesis.

In [ ]:
ljung_res = acorr_ljungbox(df['pct_change'], lags= 40, boxpierce=True)
ljung_res.head()

## Time Series Plot Summary

In [ ]:
def ts_plot(residuals, stan_residuals, lags=50):
    residuals.plot(title='GARCH Residuals', figsize=(15, 10))
    plt.show()
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(20, 10))
    ax[0].set_title('GARCH Standardized Residuals KDE')
    ax[1].set_title('GARCH Standardized Resduals Probability Plot')    
    residuals.plot(kind='kde', ax=ax[0])
    probplot(stan_residuals, dist='norm', plot=ax[1])
    plt.show()
    acf = plot_acf(stan_residuals, lags=lags)
    pacf = plot_pacf(stan_residuals, lags=lags, method="ywm")
    acf.suptitle('GARCH Model Standardized Residual Autocorrelation', fontsize=20)
    acf.set_figheight(5)
    acf.set_figwidth(15)
    pacf.set_figheight(5)
    pacf.set_figwidth(15)
    plt.show()

In [ ]:
garch = arch_model(df['pct_change'], vol='GARCH', p=1, q=1, dist='normal')
fgarch = garch.fit(disp='off') 
resid = fgarch.resid
st_resid = np.divide(resid, fgarch.conditional_volatility)
ts_plot(resid, st_resid)
fgarch.summary()

In [ ]:
arch_test = het_arch(resid, nlags=50)
shapiro_test = shapiro(st_resid)

print(f'Lagrange mulitplier p-value: {arch_test[1]}')
print(f'F test p-value: {arch_test[3]}')
print(f'Shapiro-Wilks p-value: {shapiro_test[1]}')

# GARCH model
Generalized Auto Regressive Conditional Heteroskedasticity <br>
GARCH(1,1) is a widely used econmetric model to estimate historical volatility.

GARCH estimates the historical volatility in Day $n$ with:
- **Rate of return** in Day n-1, denoted by $u$
- **Volatiltiy** in Day n-1, denoted by $\sigma$ ($\sigma^2$ is variance)

The three coefficients in the model:
- **alpha**, coefficient of $u^2$
- **beta**, coefficient of $\sigma^2$
- **omega**, constant <br>

The formula for estiamting historical volatility is:
$$\sigma^2_n = \omega + \alpha u^2_{n-1} + \beta \sigma^2_{n-1}$$


## Model Evaluation

To measure how well our ARCH/GARCH model fit our data:
1. Autocorrelation in the standardized residuals using Ljung-Box test.
2. ARCH effects (conditional heteroskedasticity) in the residuals using Engle's ARCH test on the residuals.
3. Normal distribution in the standardized residuals, we can use the Shapiro-Wilk test, Q-Q plot, and if larger n- the Jarque-Bera test to see if our data approaches normality.

### Engle's ARCH test
Deterin whether our ARCH model has captured the conditional heteroskedasticity of our time series.
- $H_o$ : The squared residuals are a sequence of white noise- the residuals are homoscedastic.
- $H_a$ : The squared residuals could not be fitted with a linear regression model and exhibit heteroskedasticity.

### Shapiro-Wilks test
Evaluates a data sample and quantifies how likely the data was drawn from a Gaussian distribution.
- $H_o$ : The data is normally distributed
- $H_a$: The data is not normally distributed

### Jarque-Bera Test
A type of lagrange multiplier test for normality. Usually used for large data sets because other normality tests are not reliable when n is large, Shapiro-Wilk is not reliable with n more than 2,000. Jarque-Bera specifically matches skewness and kurtosis to a normal distribution.
- $H_o$ : The data is normally distributed
- $H_a$: The data is not normally distributed

## Grid Search
Parameter tuning

In [ ]:
def gridsearch(data, p_rng, q_rng):
    top_score, top_results = float('inf'), None
    top_models = []
    for p in p_rng:
        for q in q_rng:
            try:
                model = arch_model(data, vol='GARCH', p=p, q=q, dist='normal')
                model_fit = model.fit(disp='off')
                resid = model_fit.resid
                st_resid = np.divide(resid, model_fit.conditional_volatility)
                results = evaluate_model(resid, st_resid)
                results['AIC'] = model_fit.aic
                results['params']['p'] = p
                results['params']['q'] = q
                if results['AIC'] < top_score: 
                    top_score = results['AIC']
                    top_results = results
                elif results['LM_pvalue'][1] is False:
                    top_models.append(results)
            except:
                continue
    top_models.append(top_results)
    return top_models

## Model Evaluation

In [ ]:
def evaluate_model(residuals, st_residuals, lags=50):
    results = {
        'LM_pvalue': None,
        'F_pvalue': None,
        'SW_pvalue': None,
        'AIC': None,
        'params': {'p': None, 'q': None}
    }
    arch_test = het_arch(residuals, nlags=lags)
    shap_test = shapiro(st_residuals)
    # We want falsey values for each of these hypothesis tests
    results['LM_pvalue'] = [arch_test[1], arch_test[1] < .05]
    results['F_pvalue'] = [arch_test[3], arch_test[3] < .05]
    results['SW_pvalue'] = [shap_test[1], shap_test[1] < .05]
    return results

In [ ]:
p_rng = range(0,30)
q_rng = range(0,40)
df['dif_pct_change'] = df['pct_change']#.diff()

# Time the grid search
start = timeit.default_timer()

top_models = gridsearch(df['dif_pct_change'], p_rng, q_rng)

stop = timeit.default_timer()
print('Time: ', stop - start)

print(top_models)

In [ ]:
p = top_models[0]['params']['p']
q = top_models[0]['params']['q']
garch = arch_model(df['pct_change'], vol='GARCH', p=p, q=q, dist='normal')
fgarch = garch.fit(disp='off') 
resid = fgarch.resid
st_resid = np.divide(resid, fgarch.conditional_volatility)
ts_plot(resid, st_resid)
arch_test = het_arch(resid, nlags=50)
shapiro_test = shapiro(st_resid)
print(f'Lagrange mulitplier p-value: {arch_test[1]}')
print(f'F test p-value: {arch_test[3]}')
print(f'Shapiro-Wilks p-value: {shapiro_test[1]}')
fgarch.summary()

# Predict Stock Volatilty

In [ ]:
garch_model = arch_model(df['pct_change'], p = p, q = q,
                      mean = 'constant', vol = 'GARCH', dist = 'normal')

gm_result = garch_model.fit(disp='off')
print(gm_result.params)

print('\n')

gm_forecast = gm_result.forecast(horizon = 5)
print(gm_forecast.variance[-1:])

In [ ]:
rolling_predictions = []
test_size = 365
for i in range(test_size):
    train = df['pct_change'][:-(test_size-i)]
    model = arch_model(train, p=p, q=q)
    model_fit = model.fit(disp='off')
    pred = model_fit.forecast(horizon=1)
    rolling_predictions.append(np.sqrt(pred.variance.values[-1,:][0]))
    
rolling_predictions = pd.Series(rolling_predictions, index=df['pct_change'].index[-365:])

plt.figure(figsize=(10,4))
plt.plot(rolling_predictions)
plt.title('Rolling Prediction')
plt.show()

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(abs(df['pct_change'][-365:]))
plt.plot(rolling_predictions)
plt.axhline(y=threshold, color='g', linestyle='-')
plt.title('Volatility Prediction - Rolling Forecast')
plt.legend(['True Voltatilty', 'Predicted Volatility'])
plt.show()

# INTEGRATE With DEEP LEARNING
AFTER UP or DOWN model
Model Stacking <br>
Ensemble stacking

TODO: Running on the 200-300 stocks <br>

Relationship between volatiltiy between up and down <br>
Up = low voltatilty, down = high volatility <br>

Consider performance on different stocks, may have to choose which stock this model is good at.

TODO: LSTM /BILSTM

# Performance Metric

In [ ]:
y_pred = np.array(rolling_predictions >= 2)
y_true = np.array(abs(df['pct_change'][-365:]) >= 2)

# Confusion Matrix
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
print("tn:", tn, "fp:", fp, "fn:", fn, "tp:", tp)

# Precision score
precision_macro = precision_score(y_true, y_pred, average='macro')
precision_micro = precision_score(y_true, y_pred, average='micro')
print("Macro precision score:", precision_macro)
print("Micro precision score:", precision_micro)

# f1 score
f1_macro = f1_score(y_true, y_pred, average='macro')
f1_micro = f1_score(y_true, y_pred, average='micro')
print("Macro f1 score:", f1_macro)
print("Micro f1 score:", f1_micro)

# Performance on multiple stocks

## TXT

In [ ]:
filename = f'reports/GARCH performance on stocks and etfs'
with open(filename, 'a') as f:
    f.write(f"\n{stock_symbol}:\nMacro precision score: {precision_macro}\nMicro precision score: {precision_micro}\nMacro f1 score: {f1_macro}\nMicro f1 score: {f1_micro}\n")

## CSV

In [ ]:
garch_performance = pd.read_csv("reports/GARCH_performance.csv")
garch_performance = garch_performance.drop(["Unnamed: 0"], axis=1)
garch_performance

In [ ]:
garch_performance.loc[len(garch_performance.index)] = [stock_symbol, precision_macro, precision_micro, f1_macro, f1_micro]
garch_performance

In [ ]:
garch_performance.to_csv("reports/GARCH_performance.csv")

# Work Cited
Liu, Wing Ki, and Mike K. P. So. "A GARCH Model with Artifical Neural Networks." 20 Oct 2020.